# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities  = pd.read_csv('clean city data')


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = cities[['Lat', 'Long']]
weights = cities['Humidity']
fig = gmaps.figure(map_type="TERRAIN")
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))




fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#find temp at and above 21˚Celius
low_temp = cities.loc[cities['Max Temp']>=21]
# low_temp.head(2)

In [5]:
#find temp at and below 27˚Celius
high_temp = low_temp.loc[low_temp['Max Temp']<=27]
# high_temp

In [6]:
#find wind speed less than equal to 1 (m/s)
low_wind = high_temp.loc[high_temp['Wind Speed']<=1]
# low_wind

In [7]:
#it seems like it was avery cloudy day
# Store into variable named hotel_df.
hotel_df = low_wind.loc[low_wind['Cloudiness']<=90]
hotel_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,Fairbanks,64.84,-147.72,25.56,50,1,0.78,US,1596319690
45,Dum Duma,27.57,95.57,25.15,95,86,0.32,IN,1596319683
178,Ryōtsu-minato,38.08,138.43,22.00,100,50,1.00,JP,1596319725
192,Honiara,-9.43,159.95,25.00,88,75,0.49,SB,1596319689
226,Ürümqi,43.80,87.60,23.00,35,0,1.00,CN,1596319737
395,Tiznit Province,29.58,-9.50,26.76,44,20,0.61,MA,1596319778
409,Ankang,32.68,109.02,24.03,86,76,0.89,CN,1596319782
411,Mannārakkāt,10.98,76.47,23.00,94,75,1.00,IN,1596319782
484,Shimoda,34.67,138.95,22.78,86,23,0.45,JP,1596319800
489,Yining,29.04,114.56,25.31,86,74,0.77,CN,1596319720


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df.reset_index(drop=True, inplace=True)
hotel_df.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Fairbanks,64.84,-147.72,25.56,50,1,0.78,US,1596319690
1,Dum Duma,27.57,95.57,25.15,95,86,0.32,IN,1596319683
2,Ryōtsu-minato,38.08,138.43,22.00,100,50,1.00,JP,1596319725
3,Honiara,-9.43,159.95,25.00,88,75,0.49,SB,1596319689
4,Ürümqi,43.80,87.60,23.00,35,0,1.00,CN,1596319737


In [9]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df.insert(1,'Hotel Name','')
# cost_tbl.insert(1, "col_name", "") 
hotel_df

,City,Hotel Name,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Fairbanks,,64.84,-147.72,25.56,50,1,0.78,US,1596319690
1,Dum Duma,,27.57,95.57,25.15,95,86,0.32,IN,1596319683
2,Ryōtsu-minato,,38.08,138.43,22.00,100,50,1.00,JP,1596319725
3,Honiara,,-9.43,159.95,25.00,88,75,0.49,SB,1596319689
4,Ürümqi,,43.80,87.60,23.00,35,0,1.00,CN,1596319737
5,Tiznit Province,,29.58,-9.50,26.76,44,20,0.61,MA,1596319778
6,Ankang,,32.68,109.02,24.03,86,76,0.89,CN,1596319782
7,Mannārakkāt,,10.98,76.47,23.00,94,75,1.00,IN,1596319782
8,Shimoda,,34.67,138.95,22.78,86,23,0.45,JP,1596319800
9,Yining,,29.04,114.56,25.31,86,74,0.77,CN,1596319720


In [10]:
params = {
    "radius": 5000, 
    "type": "lodging", 
    "key": g_key
}


for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Long']
    
    params["location"] = f"{lat},{lng}"
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()

#     print(json.dumps(name_address, indent=4, sort_keys= True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print("missing----skip")
    

/Users/darenhamilton/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


missing----skip


In [11]:
hotel_df

,City,Hotel Name,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Fairbanks,Westmark Fairbanks Hotel & Conference Center,64.84,-147.72,25.56,50,1,0.78,US,1596319690
1,Dum Duma,Komal bakery shope,27.57,95.57,25.15,95,86,0.32,IN,1596319683
2,Ryōtsu-minato,Sadokamoko Onsen Kohanno Yado Yoshidaya,38.08,138.43,22.00,100,50,1.00,JP,1596319725
3,Honiara,Heritage Park Hotel,-9.43,159.95,25.00,88,75,0.49,SB,1596319689
4,Ürümqi,Hua Ling Grand Hotel Urumqi,43.80,87.60,23.00,35,0,1.00,CN,1596319737
5,Tiznit Province,,29.58,-9.50,26.76,44,20,0.61,MA,1596319778
6,Ankang,安琪酒店,32.68,109.02,24.03,86,76,0.89,CN,1596319782
7,Mannārakkāt,The Park Residency,10.98,76.47,23.00,94,75,1.00,IN,1596319782
8,Shimoda,Shimoda Tokyu Hotel,34.67,138.95,22.78,86,23,0.45,JP,1596319800
9,Yining,Xiushui Hotel,29.04,114.56,25.31,86,74,0.77,CN,1596319720


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))